# Task 1: Data Collection and Preparation
### Student Name: Edmond Agbaje
### Student Number: 19366741
##### Stocks Chosen: JPMorgan Chase, Moderna Inc., Delta Air Lines, Amazon.com Inc.

 This notebook collects the recent stocks of 4 companies JPMorgan Chase, Moderna Inc., Delta Air Lines, Amazon.com Inc.

In [1]:
import json, requests, urllib
from pathlib import Path
from datetime import datetime
import pandas as pd
import bs4
import pickle

Settings for the HTML and data collection.

In [2]:
#html prefixes are the same for all
html_prefix = "http://mlg.ucd.ie/modules/COMP30760/stocks/data-"
#html for each company
company_dic = {"Amazon":"amzn.html","Delta Air Lines":"dal.html", "JPMorgan":"jpm.html", "Moderna": "mrna.html"}
#dictionary for the stock tickers 
company_ids = {"Amazon":"AMZN", "Delta Air Lines": "DAL", "JPMorgan": "JPM", "Moderna": "MRNA"}
#column names for data frames
columns = ["Open","High","Low","Close"]

# Collecting Data

Define a function to fetch stock data from HTML.

In [3]:
def fetch(link):
    #fetching data from HTML code on the web page
    response = urllib.request.urlopen(html_prefix + link)
    html = response.read().decode()
    
    return html

Now we need to parse this raw data.
Create function to parse this data.

In [4]:
def parse_data(data,stock):
    #use Beautiful soup to find all the "td" tags and collect the text in them
    parser = bs4.BeautifulSoup(data,"html.parser")
    row = 0
    count =0 
    #2-D array to collect all the data for all companies 
    df = [[] for i in (range(522))]
    for match in parser.find_all("td"):
        text = match.get_text()
        df[row].append(text)
        count+=1
        #split the data to a new row at every eigthth column as each row has eigth values
        if count == 8:
            row += 1
            count = 0
    
    #create the data frame for the stocks of each company
    pdData = pd.DataFrame(df, columns = ["Stock","Day", "Month","Year","Open","High","Low","Close"])
    return pdData

In [5]:
df_stock = {}
#check to see that each company's stocks has been fetched
for stock in company_ids:
    df = fetch(company_dic[stock])
    df_stock[stock] = parse_data(df,company_ids[stock])
    print (company_ids[stock])

AMZN
DAL
JPM
MRNA


# Data Verification

Check that the data frames are correctly created.

In [6]:
df_stock["Amazon"].head()

,Stock,Day,Month,Year,Open,High,Low,Close
0,Stock,Day,Month,Year,Open,High,Low,Close
1,AMZN,01,Oct,2019,1746.000000,1755.599976,1728.410034,1735.650024
2,AMZN,02,Oct,2019,1727.739990,1728.890015,1705.000000,1713.229980
3,AMZN,03,Oct,2019,1713.000000,1725.000000,1685.060059,1724.420044
4,AMZN,04,Oct,2019,1726.020020,1740.579956,1719.229980,1739.650024


In [7]:
df_stock["Delta Air Lines"].tail()

,Stock,Day,Month,Year,Open,High,Low,Close
517,DAL,24,Sep,2021,42.840000,43.740002,42.669998,43.529999
518,DAL,27,Sep,2021,43.900002,45.060001,43.750000,43.779999
519,DAL,28,Sep,2021,43.660000,44.209999,43.310001,43.439999
520,DAL,29,Sep,2021,43.689999,43.990002,42.750000,43.360001
521,DAL,30,Sep,2021,43.060001,43.080002,42.369999,42.610001


Check that NaN values and monthly dividers were also created in the data frame.

In [8]:
df_stock["Amazon"].head(30)

,Stock,Day,Month,Year,Open,High,Low,Close
0,Stock,Day,Month,Year,Open,High,Low,Close
1,AMZN,01,Oct,2019,1746.000000,1755.599976,1728.410034,1735.650024
2,AMZN,02,Oct,2019,1727.739990,1728.890015,1705.000000,1713.229980
3,AMZN,03,Oct,2019,1713.000000,1725.000000,1685.060059,1724.420044
4,AMZN,04,Oct,2019,1726.020020,1740.579956,1719.229980,1739.650024
5,AMZN,07,Oct,2019,1731.630005,1747.829956,1723.699951,1732.660034
6,AMZN,08,Oct,2019,1722.489990,1727.000000,1705.000000,1705.510010
7,AMZN,09,Oct,2019,1719.609985,1729.949951,1714.359985,1721.989990
8,AMZN,10,Oct,2019,1725.239990,1738.290039,1713.750000,1720.260010
9,AMZN,11,Oct,2019,1742.920044,1745.449951,1729.859985,1731.920044


In [9]:
def summarise_data():
    rows = []
    for stock in df_stock:
        row = {"Stock": stock}
        row["Rows"] = len(df_stock[stock])
        rows.append(row)
    return pd.DataFrame(rows).set_index("Stock")

#display the rows
df_rows = summarise_data()
df_rows

,Rows
Stock,
Amazon,522
Delta Air Lines,522
JPMorgan,522
Moderna,522


# Data Clean Up

Clean the data to just have the data we want.(Just the stocks)

Function to clean the data frame.

In [10]:
def clean(data,company):
    #enumerate through the data frame and drop the rows where the rows dont contain stock names
    for i,checker in enumerate(data["Stock"]):
        if checker != company_ids[company]:
            data = data.drop(i)
    #reset the index of the data frames 
    data = data.reset_index(drop = True)
    
    return data

Clean the data for all the company stocks.

In [11]:
for company in df_stock:
    df_stock[company] = clean(df_stock[company],company)

Verify that the data frames have been cleaned.

In [12]:
df_stock["Amazon"].head(30)

,Stock,Day,Month,Year,Open,High,Low,Close
0,AMZN,01,Oct,2019,1746.000000,1755.599976,1728.410034,1735.650024
1,AMZN,02,Oct,2019,1727.739990,1728.890015,1705.000000,1713.229980
2,AMZN,03,Oct,2019,1713.000000,1725.000000,1685.060059,1724.420044
3,AMZN,04,Oct,2019,1726.020020,1740.579956,1719.229980,1739.650024
4,AMZN,07,Oct,2019,1731.630005,1747.829956,1723.699951,1732.660034
5,AMZN,08,Oct,2019,1722.489990,1727.000000,1705.000000,1705.510010
6,AMZN,09,Oct,2019,1719.609985,1729.949951,1714.359985,1721.989990
7,AMZN,10,Oct,2019,1725.239990,1738.290039,1713.750000,1720.260010
8,AMZN,11,Oct,2019,1742.920044,1745.449951,1729.859985,1731.920044
9,AMZN,14,Oct,2019,1728.910034,1741.890015,1722.000000,1736.430054


In [13]:
#display the rows
df_rows = summarise_data()
df_rows

,Rows
Stock,
Amazon,505
Delta Air Lines,505
JPMorgan,505
Moderna,505


# Missing Values

We will now deal with the missing values("NaN") in the stocks data frames.

Function to check how many missing values are in each data frame.

In [14]:
def num_NaN(data,company):
    count=0
    for col in columns:
        for i,checker in enumerate(data[col]):
            if checker == "NaN":
                count += 1
                    
    print(company + " Stock: "+str(count) + " values missing")
    
    return data
    

In [15]:
for stock in df_stock:
    df_stock[stock] = num_NaN(df_stock[stock], stock)

Amazon Stock: 104 values missing
Delta Air Lines Stock: 96 values missing
JPMorgan Stock: 116 values missing
Moderna Stock: 112 values missing


Function to handle the missing values in the data frames.

In [16]:
def handle_missing(data,company):
    count=0
    for col in columns:
        for i,checker in enumerate(data[col]):
            #if 2 consecutive rows have missing values, replace with average of row before and row 2 after
            if checker == "NaN":
                if (data[col][i+1] == "NaN"):
                    data[col][i] = (float(data[col][i-1]) + float(data[col][i+2]))/2
                    count += 1
                else:
                    #for all other mising values, replace by getting the average of the row before and row after
                    data[col][i] = (float(data[col][i-1]) + float(data[col][i+1]))/2
                    count += 1 
                    
    print(company+ " Stock: " +str(count) + " values filled in")
    
    return data
    

In [17]:
for stock in df_stock:
    df_stock[stock] = handle_missing(df_stock[stock], stock)

Amazon Stock: 104 values filled in
Delta Air Lines Stock: 96 values filled in
JPMorgan Stock: 116 values filled in
Moderna Stock: 112 values filled in


In [18]:
df_stock["Delta Air Lines"].head(20)

,Stock,Day,Month,Year,Open,High,Low,Close
0,DAL,01,Oct,2019,58.049999,58.680000,56.650002,57.009998
1,DAL,02,Oct,2019,55.740002,55.750000,52.869999,54.349998
2,DAL,03,Oct,2019,53.119999,53.860001,51.810001,52.830002
3,DAL,04,Oct,2019,52.849998,53.959999,52.680000,53.810001
4,DAL,07,Oct,2019,53.750000,54.110001,53.240002,53.360001
5,DAL,08,Oct,2019,53.020000,54.099998,52.889999,53.200001
6,DAL,09,Oct,2019,54.009998,54.400002,53.590000,53.919998
7,DAL,10,Oct,2019,51.950001,53.259998,51.070000,53.099998
8,DAL,11,Oct,2019,52,53.23,51.52,53.045
9,DAL,14,Oct,2019,52.049999,53.200001,51.970001,52.990002


Now we need to deal with the dates by putting them into one column so it's easier to display our analysis in Task 2.

Function to adjust the columns in each data frame and change the data types of columns so they can be manipulated in Task 2.

In [19]:
def adjust_columns(stock):
    #rename the day column as date as this will be the start of the dates
    df_stock[stock] = df_stock[stock].rename(columns={"Day":"Date"})
    #add the month and year to the dates column and convert them to a string
    df_stock[stock]["Date"] = df_stock[stock]["Date"].astype(str) + "-" + df_stock[stock]["Month"].astype(str) + "-" + df_stock[stock]["Year"].astype(str)
    #delete month and year columns
    df_stock[stock] = df_stock[stock].drop(df_stock[stock].columns[[2,3]],axis=1)
    #change the date column to a datetime
    df_stock[stock]["Date"] = pd.to_datetime(df_stock[stock]["Date"])
    #change Stock,Open,High,Low and Close to str and floats so we can them in task 2 for Data Analysis
    df_stock[stock]["Stock"] = df_stock[stock]["Stock"].astype(str)
    df_stock[stock]["Open"] = df_stock[stock]["Open"].astype(float)
    df_stock[stock]["High"] = df_stock[stock]["High"].astype(float)
    df_stock[stock]["Low"] = df_stock[stock]["Low"].astype(float)
    df_stock[stock]["Close"] = df_stock[stock]["Close"].astype(float)
    #index the data frame by the Date
    df_stock[stock] = df_stock[stock].set_index("Date")
    
    print("Data Frame for " + stock + " stock has been adjusted along with column dtypes")

In [20]:
for stock in company_ids:
    adjust_columns(stock)

Data Frame for Amazon stock has been adjusted along with column dtypes
Data Frame for Delta Air Lines stock has been adjusted along with column dtypes
Data Frame for JPMorgan stock has been adjusted along with column dtypes
Data Frame for Moderna stock has been adjusted along with column dtypes


Check to see if the tables have been reformatted and that the column dtypes have been changed.

In [21]:
df_stock["Amazon"].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 505 entries, 2019-10-01 to 2021-09-30
Data columns (total 5 columns):
Stock    505 non-null object
Open     505 non-null float64
High     505 non-null float64
Low      505 non-null float64
Close    505 non-null float64
dtypes: float64(4), object(1)
memory usage: 23.7+ KB


In [22]:
df_stock["Delta Air Lines"].head(30)

,Stock,Open,High,Low,Close
Date,,,,,
2019-10-01,DAL,58.049999,58.680000,56.650002,57.009998
2019-10-02,DAL,55.740002,55.750000,52.869999,54.349998
2019-10-03,DAL,53.119999,53.860001,51.810001,52.830002
2019-10-04,DAL,52.849998,53.959999,52.680000,53.810001
2019-10-07,DAL,53.750000,54.110001,53.240002,53.360001
2019-10-08,DAL,53.020000,54.099998,52.889999,53.200001
2019-10-09,DAL,54.009998,54.400002,53.590000,53.919998
2019-10-10,DAL,51.950001,53.259998,51.070000,53.099998
2019-10-11,DAL,52.000000,53.230000,51.520001,53.045000


Now the data is cleaned and ready to be used for analysis.

# Saving Data

Saving the data frames to pickle files.

In [23]:
df_stock["Amazon"].to_pickle("Amazon.pkl")
df_stock["Delta Air Lines"].to_pickle("Delta Air Lines.pkl")
df_stock["JPMorgan"].to_pickle("JPMorgan.pkl")
df_stock["Moderna"].to_pickle("Moderna.pkl")